# Effective Pandas
Starting from Chapter 34, and progressing until the notebook length becomes challenging.  Likely 100 cells of code across a the notebook.  Initiate notebook with data sets being leveraged in prior notebooks to facilitate continuation.

- Car data notebook with mpg data
- World cup history datasource
- Snow resort precipitation records
- Siena President evaluation data
- Jetbrains survey data

### Imports
Minimum imports to follow allong are pandas and numpy.  Including matplotlib.  Unclear if this is needed at this time.

### Tweak Functions
Functions developed previously to optimize data will be leveraged to reset existing data to lower memory usage systems leveraging chaining system used in Chapter 21.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import catboost as cb
import collections

Bad value in file WindowsPath('C:/Users/KNHOFFM/.matplotlib/stylelib/xom.mplstyle'), line 11 ('lines.color: #0C479D'): Key lines.color: '' does not look like a color arg
Bad value in file WindowsPath('C:/Users/KNHOFFM/.matplotlib/stylelib/xom.mplstyle'), line 15 ('patch.edgecolor: #FFFFFF'): Key patch.edgecolor: '' does not look like a color arg
Bad value in file WindowsPath('C:/Users/KNHOFFM/.matplotlib/stylelib/xom.mplstyle'), line 16 ('patch.facecolor: #0C479D'): Key patch.facecolor: '' does not look like a color arg


### Datasets

In [2]:
cars = pd.read_csv("vehicles.csv")

C:\Users\KNHOFFM\AppData\Local\Temp\ipykernel_7336\2339704932.py:1: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  cars = pd.read_csv("vehicles.csv")


In [3]:
fifa = pd.read_csv("international_matches.csv")

In [4]:
alta_df = pd.read_csv('alta-noaa-1980-2019.csv')

In [5]:
pres_df = pd.read_csv('siena2018-pres.csv', index_col=0)

In [6]:
jb = pd.read_csv('2020-jetbrains-python-survey.csv') # normally pursue url

C:\Users\KNHOFFM\AppData\Local\Temp\ipykernel_7336\763600581.py:1: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv('2020-jetbrains-python-survey.csv') # normally pursue url


In [7]:
df = pd.read_csv("dirtydevil.txt", skiprows=lambda num: num < 34 or num == 35, sep = '\t') 

C:\Users\KNHOFFM\AppData\Local\Temp\ipykernel_7336\3445993392.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dirtydevil.txt", skiprows=lambda num: num < 34 or num == 35, sep = '\t')


### Tweak Dataset optimization
Pulled from prior sessions to prep datasets in advance of further analysis and interogation.  This will be pulled where already developed and loaded into this notebook.
- World Cup data set
- Siena Presidents
- Jetbrains survey

### JetBrains Dataset tweak
---

In [8]:
def get_uniu_cols(jb): 
    # jb here is to help see the passed data, but not technically require as it confuses the name space
    '''This function is unique to this data set due to the field structure using periods and having duplicate fields.
    It strips out the variants of a column, and targets the root fields for the dataset'''
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols

In [9]:
def prep_for_ml(df):
    # remove pandas specific dtypes
    '''Specifically done to help with the use of catboost, which requires base Python types'''
    return (df
            .assign(**{col:df[col].astype(float) for col in df.select_dtypes('number')},
                    **{col:df[col].astype(str).fillna('') for col in df.select_dtypes(['object', 'category'])})
           )

In [10]:
def predict_col(df, col):
    '''leverages the prep_for_ml return value to conduct prediction on the series and return a series with
    missing data imputed based on the prediction engine'''
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i, typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = (missing.drop(columns=[col]).values)
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [11]:
def tweak_jb(jb):
    '''Full clean up and modification of jb dataframe from jetbrains survey response'''
    uniq_cols = get_uniu_cols(jb)
    return (jb
            [uniq_cols]
            .rename(columns=lambda c:c.replace('.','_'))
            .assign(age = lambda df_:df_.age.str[:2].astype(float).astype('Int64'),
                    are_you_datascientist = lambda df_: df_.are_you_datascientist
                        .replace({'Yes':True,'No':False,np.nan:False}),
                    company_size = lambda df_:df_.company_size.replace({'51–500':51,'More than 5,000':5000,
                        '11–50':11,'2–10':2,'1,001–5,000':1001,'Just me':1,'501–1,000':501,'Not sure':np.nan})
                        .astype('Int64'),
                    country_live = lambda df_:df_.country_live.astype('category'),
                    employment_status = lambda df_:df_.employment_status.fillna('Other').astype('category'),
                    is_python_main = lambda df_:df_.is_python_main.astype('category'),
                    team_size = lambda df_: df_.team_size.str.split(r'-', n=1, expand=True)
                        .iloc[:,0].replace('More than 40 people',41)
                        .where(df_.company_size!=1, 1).astype(float),
                    years_of_coding = lambda df_: df_.years_of_coding.replace('Less than 1 year',0.5)
                        .str.extract(r'(\d+)').astype(float),
                    python3_ver = lambda df_:df_.python3_version_most.str.replace('_','.')
                        .str.extract(r'(\d\.\d)').astype(float),
                    use_python_most = lambda df_: df_.use_python_most.fillna('Unknown')
                   )
            .assign(team_size = lambda df_: predict_col(df_, 'team_size').astype(int))
            .drop(columns=['python2_version_most'])
            .dropna()
           )

In [12]:
jb = tweak_jb(jb)

Learning rate set to 0.5
0:	learn: 2.9835208	total: 212ms	remaining: 4.02s
1:	learn: 2.8774954	total: 272ms	remaining: 2.44s
2:	learn: 2.8394639	total: 324ms	remaining: 1.83s
3:	learn: 2.8050221	total: 435ms	remaining: 1.74s
4:	learn: 2.7869339	total: 583ms	remaining: 1.75s
5:	learn: 2.7743464	total: 639ms	remaining: 1.49s
6:	learn: 2.7620047	total: 714ms	remaining: 1.32s
7:	learn: 2.7579574	total: 762ms	remaining: 1.14s
8:	learn: 2.7499983	total: 806ms	remaining: 985ms
9:	learn: 2.7459189	total: 852ms	remaining: 852ms
10:	learn: 2.7455531	total: 905ms	remaining: 740ms
11:	learn: 2.7417426	total: 948ms	remaining: 632ms
12:	learn: 2.7409715	total: 988ms	remaining: 532ms
13:	learn: 2.7397709	total: 1.03s	remaining: 443ms
14:	learn: 2.7329598	total: 1.08s	remaining: 361ms
15:	learn: 2.7271657	total: 1.14s	remaining: 284ms
16:	learn: 2.7223690	total: 1.22s	remaining: 215ms
17:	learn: 2.7209157	total: 1.27s	remaining: 141ms
18:	learn: 2.7162403	total: 1.32s	remaining: 69.4ms
19:	learn: 2.71

### Siena Dataset tweak
---

In [13]:
def tweak_siena(df):
    """Convert Siena presidential Dataframe into a more usable format with appropriate dtypes"""
    def int64_to_uint8(df_):
        """convert int columns to uint8 dtype for space/memory"""
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols})
               )
    return (df
            .rename(columns={'Seq.':'Seq'}) #1 column name update
            .rename(columns={k:v.replace(' ','_') for k,v in
                {'Bg':'Background',
                 'PL':'Party Leadership','CAb':'Communication ability',
                 'RC':'Relations with Congress','CAp':'Court appointments',
                 'HE':'Handling of economy','L':'Luck',
                 'AC':'Ability to compromise','WR':'Willing to take risks',
                 'EAp':'Executive appointments','OA':'Overall Ability',
                 'Im':'Imagination','DA':'Domestic accomplishments',
                 'Int':'Integrity','EAb':'Executive ability',
                 'FPA':'Foriegn policy accomplishments',
                 'LA':'Leadership ability',
                 'IQ':'Intelligence','AM':'Avoid crucial mistakes',
                 'EV':'Experts view','O':'Overall'}.items()})
            .astype({'Party':'category'}) #2 category convert
            .pipe(int64_to_uint8) #3 dtypes
            .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8') #4 create new columns
                            .sum(axis=1).rank(method='dense').astype('uint8')),
                   Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                        labels='Ist 2nd 3rd 4th'.split())
                   )
           )

In [14]:
pres = tweak_siena(pres_df)

### Fifa Dataset Tweak
---

In [15]:
def tweak_fifa(fifa):
    return (fifa
            .assign(date = lambda df_:pd.to_datetime(df_.date),
                    home_team = lambda df_:df_.home_team.astype('category'),
                    away_team = lambda df_:df_.away_team.astype('category'),
                    home_team_continent = lambda df_:df_.home_team_continent.astype('category'),
                    away_team_continent = lambda df_:df_.away_team_continent.astype('category'),
                    home_team_fifa_rank = lambda df_:df_.home_team_fifa_rank.astype('int32'),
                    away_team_fifa_rank = lambda df_:df_.away_team_fifa_rank.astype('int32'),
                    home_team_total_fifa_points = lambda df_:df_.home_team_total_fifa_points.astype('int32'),
                    away_team_total_fifa_points = lambda df_:df_.away_team_total_fifa_points.astype('int32'),
                    home_team_score = lambda df_:df_.home_team_score.astype('int8'),
                    away_team_score = lambda df_:df_.away_team_score.astype('int8'),
                    tournament = lambda df_:df_.tournament.astype('category'),
                    city = lambda df_:df_.city.astype('category'),
                    country = lambda df_:df_.country.astype('category'),
                    shoot_out = lambda df_: df_.shoot_out.where(df_.shoot_out=='Yes',True).where(df_.shoot_out=='No',False),
                    home_team_result = lambda df_:df_.home_team_result.astype('category'))
           )

In [16]:
fifa = tweak_fifa(fifa)

### Dirty Devil River Dataset Tweak
---

In [17]:
def tweak_river(df_):
    return (df_
            .assign(datetime = pd.to_datetime(df_.datetime))
            .rename(columns = {'144166_00060':'cfs',
                               '144167_00065':'gage_height'})
            .set_index('datetime')
           )

In [18]:
dd = tweak_river(df)

## Chapter 34 - Styling Dataframes
Styling dataframes is an often underutilized data inference process which can assist in interogation of data during EDA or troubleshooting.  

### Data source
This section of the book scope will leverage a data set from teh US Geologic Survey associated with river flow on the Dirty Devil River in Utah.  The data set was loaded and an initial tweak function was applied already.

In [19]:
dd.head(2)

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00,USGS,9333500,MDT,71.0,A:[91],NaN,NaN
2001-05-07 01:15:00,USGS,9333500,MDT,71.0,A:[91],NaN,NaN


#### Aggregation and Column addition
With the data processed and available, the cell below will aggregate the data and add columns to obtain a view of the information which will assist in the analysis and provide a better summary level for the dataframe styling processes.  

>**Note:** Resample does not support named aggregations, which are tuples with the column name and the subsequent aggregation.  The pd.Grouper() method does allow this approach and was used by the tutorial.  This new column creation would not have been easy or possible in a resample approach.

In [20]:
import sparklines

In [21]:
agg_flow = (dd
            .groupby(pd.Grouper(freq='M'))
            .agg(cfs = ('cfs', 'median'),
                 total_flow = ('cfs', lambda ser: (ser*15*60).sum()),
                 gage_height = ('gage_height', 'median'),
                 flow_trend = ('cfs', lambda ser: sparklines.sparklines(
                     ser
                     .fillna(0)
                     .resample('2D')
                     .median()
                     .fillna(0))
                     [0])
                )
            .assign(quarterly_flow = lambda df: df.total_flow.resample('Q').transform('sum'),
                    percent_quarterly_flow = lambda df: df.total_flow/df.quarterly_flow,
                    off_goal = lambda df: df.percent_quarterly_flow - 0.33,
                    cost = lambda df: df.total_flow*0.0002)
)

In [22]:
agg_flow.tail()

,cfs,total_flow,gage_height,flow_trend,quarterly_flow,percent_quarterly_flow,off_goal,cost
datetime,,,,,,,,
2020-05-31,21.25,60721029.0,6.51,▇▅▇▆▅▆█▆▄▃▃▃▃▂▂▁,262374219.0,0.231429,-0.098571,12144.2058
2020-06-30,10.20,24475410.0,6.28,▄▅▅▅██▆▄▁▁▁▁▁▁▁,262374219.0,0.093284,-0.236716,4895.0820
2020-07-31,10.80,67073337.0,6.05,▁▁▁▁▂▂▂▂▂▂▂▂█▅█▅,88485345.0,0.758016,0.428016,13414.6674
2020-08-31,0.32,11042316.0,5.48,█▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁,88485345.0,0.124793,-0.205207,2208.4632
2020-09-30,5.79,10369692.0,6.01,▁▁▁▁▁▁▅▅▅▅▆▇██,88485345.0,0.117191,-0.212809,2073.9384


In [23]:
(agg_flow
 .reset_index()
 .head(10)
 .style
 .format({'cost':'${:,.2f}','datetime':'{:%Y/%m}/01', 'percent_quarterly_flow':'{:.1%}','off_goal':'{:+.1%}',
         **{col:'{:.1f}' for col in ['cfs','quarterly_flow','total_flow']}},
         na_rep='Missing')
)

,datetime,cfs,total_flow,gage_height,flow_trend,quarterly_flow,percent_quarterly_flow,off_goal,cost
0,2001/05/01,47.0,105383700.0,Missing,█▅▅▃▅▄▄▃▂▃▂▁▁,123227100.0,85.5%,+52.5%,"$21,076.74"
1,2001/06/01,23.0,17843400.0,Missing,█▇▆▄▁▁▁▁▁▁▁▁▁▁▆,123227100.0,14.5%,-18.5%,"$3,568.68"
2,2001/07/01,17.0,7781400.0,Missing,▁█▄,243448920.0,3.2%,-29.8%,"$1,556.28"
3,2001/08/01,52.5,192848220.0,Missing,█▂▁▁▁,243448920.0,79.2%,+46.2%,"$38,569.64"
4,2001/09/01,26.0,42819300.0,Missing,▆▆▁▁▅▆▁██▆▆▆▆,243448920.0,17.6%,-15.4%,"$8,563.86"
5,2001/10/01,54.0,134975700.0,Missing,▁▁▂▂▃▄▄▅▅▅▆▇▇▇█,480483900.0,28.1%,-4.9%,"$26,995.14"
6,2001/11/01,104.0,203110200.0,Missing,▆▆▆▆▆▇▇▇██▁▁▁▇▇,480483900.0,42.3%,+9.3%,"$40,622.04"
7,2001/12/01,115.0,142398000.0,Missing,▆▁▁▁▇▇▁▁▁▆▆▆▁▆▇█,480483900.0,29.6%,-3.4%,"$28,479.60"
8,2002/01/01,136.0,197745300.0,Missing,██▇███▇▇▁▁▁▁▁▁▇,638525700.0,31.0%,-2.0%,"$39,549.06"
9,2002/02/01,131.0,157920300.0,Missing,▄▄█▆▆█▁,638525700.0,24.7%,-8.3%,"$31,584.06"


In [24]:
(agg_flow
 .reset_index()
 .head(10)
 .style
 .format({'cost':'${:,.2f}','datetime':'{:%Y/%m}/01', 'percent_quarterly_flow':'{:.1%}','off_goal':'{:+.1%}',
         **{col:'{:.1f}' for col in ['cfs','quarterly_flow','total_flow']}},
         na_rep='Missing')
 .bar(subset='cfs', color='#c07fef', vmax=agg_flow.cfs.quantile(.95))
 .bar(subset='off_goal', color=['red','green'], align='mid')
 .highlight_null(color='##fef70c')
 .highlight_max(axis=0, color='green')
)

,datetime,cfs,total_flow,gage_height,flow_trend,quarterly_flow,percent_quarterly_flow,off_goal,cost
0,2001/05/01,47.0,105383700.0,Missing,█▅▅▃▅▄▄▃▂▃▂▁▁,123227100.0,85.5%,+52.5%,"$21,076.74"
1,2001/06/01,23.0,17843400.0,Missing,█▇▆▄▁▁▁▁▁▁▁▁▁▁▆,123227100.0,14.5%,-18.5%,"$3,568.68"
2,2001/07/01,17.0,7781400.0,Missing,▁█▄,243448920.0,3.2%,-29.8%,"$1,556.28"
3,2001/08/01,52.5,192848220.0,Missing,█▂▁▁▁,243448920.0,79.2%,+46.2%,"$38,569.64"
4,2001/09/01,26.0,42819300.0,Missing,▆▆▁▁▅▆▁██▆▆▆▆,243448920.0,17.6%,-15.4%,"$8,563.86"
5,2001/10/01,54.0,134975700.0,Missing,▁▁▂▂▃▄▄▅▅▅▆▇▇▇█,480483900.0,28.1%,-4.9%,"$26,995.14"
6,2001/11/01,104.0,203110200.0,Missing,▆▆▆▆▆▇▇▇██▁▁▁▇▇,480483900.0,42.3%,+9.3%,"$40,622.04"
7,2001/12/01,115.0,142398000.0,Missing,▆▁▁▁▇▇▁▁▁▆▆▆▁▆▇█,480483900.0,29.6%,-3.4%,"$28,479.60"
8,2002/01/01,136.0,197745300.0,Missing,██▇███▇▇▁▁▁▁▁▁▇,638525700.0,31.0%,-2.0%,"$39,549.06"
9,2002/02/01,131.0,157920300.0,Missing,▄▄█▆▆█▁,638525700.0,24.7%,-8.3%,"$31,584.06"


In [26]:
(agg_flow
 .reset_index()
 .iloc[:30]
 .style
 .format({'cost':'${:,.2f}','datetime':'{:%Y/%m}/01', 'percent_quarterly_flow':'{:.1%}','off_goal':'{:+.1%}',
         **{col:'{:.1f}' for col in ['cfs','quarterly_flow','total_flow']}},
         na_rep='Missing')
 .bar(subset='cfs', color='#c07fef', vmax=agg_flow.cfs.quantile(.95))
 .bar(subset='off_goal', color=['red','green'], align='mid')
 .highlight_null(color='#fef70c', subset='gage_height')
 .highlight_max(axis=0, color='green')
 .background_gradient(axis=0, cmap='Blues', subset='cost', vmin=agg_flow.cost.min(), vmax=agg_flow.cost.quantile(0.90))
 .set_caption('Dirty Devil Summary')
)

,datetime,cfs,total_flow,gage_height,flow_trend,quarterly_flow,percent_quarterly_flow,off_goal,cost
0,2001/05/01,47.0,105383700.0,Missing,█▅▅▃▅▄▄▃▂▃▂▁▁,123227100.0,85.5%,+52.5%,"$21,076.74"
1,2001/06/01,23.0,17843400.0,Missing,█▇▆▄▁▁▁▁▁▁▁▁▁▁▆,123227100.0,14.5%,-18.5%,"$3,568.68"
2,2001/07/01,17.0,7781400.0,Missing,▁█▄,243448920.0,3.2%,-29.8%,"$1,556.28"
3,2001/08/01,52.5,192848220.0,Missing,█▂▁▁▁,243448920.0,79.2%,+46.2%,"$38,569.64"
4,2001/09/01,26.0,42819300.0,Missing,▆▆▁▁▅▆▁██▆▆▆▆,243448920.0,17.6%,-15.4%,"$8,563.86"
5,2001/10/01,54.0,134975700.0,Missing,▁▁▂▂▃▄▄▅▅▅▆▇▇▇█,480483900.0,28.1%,-4.9%,"$26,995.14"
6,2001/11/01,104.0,203110200.0,Missing,▆▆▆▆▆▇▇▇██▁▁▁▇▇,480483900.0,42.3%,+9.3%,"$40,622.04"
7,2001/12/01,115.0,142398000.0,Missing,▆▁▁▁▇▇▁▁▁▆▆▆▁▆▇█,480483900.0,29.6%,-3.4%,"$28,479.60"
8,2002/01/01,136.0,197745300.0,Missing,██▇███▇▇▁▁▁▁▁▁▇,638525700.0,31.0%,-2.0%,"$39,549.06"
9,2002/02/01,131.0,157920300.0,Missing,▄▄█▆▆█▁,638525700.0,24.7%,-8.3%,"$31,584.06"


In [45]:
(agg_flow
 .reset_index()
 .iloc[:30]
 .style
 .format({'cost':'${:,.2f}','datetime':'{:%Y/%m}/01', 'percent_quarterly_flow':'{:.1%}','off_goal':'{:+.1%}',
         **{col:'{:.1f}' for col in ['cfs','quarterly_flow','total_flow']}},
         na_rep='Missing')
 .bar(subset='cfs', color='#c07fef', vmax=agg_flow.cfs.quantile(.95))
 .bar(subset='off_goal', color=['red','green'], align='mid')
 .highlight_null(color='#fef70c', subset='gage_height')
 .highlight_max(axis=0, color='green')
 .background_gradient(axis=0, cmap='Blues', subset='cost', vmin=agg_flow.cost.min(), vmax=agg_flow.cost.quantile(0.90))
 .set_caption('Dirty Devil Summary')
 .applymap(lambda val: f'color: "grey"; opacity:80%; background-color:{"#4580ae" if val > 0 else "#c07fef"}', 
            subset='cfs')
 .set_table_styles([{'selector':'td:hover', 'props':'border:3px solid lightgreen; font-size:10pt;'}])
 .set_sticky(axis="columns")
 .hide(axis='index')
)

datetime,cfs,total_flow,gage_height,flow_trend,quarterly_flow,percent_quarterly_flow,off_goal,cost
2001/05/01,47.0,105383700.0,Missing,█▅▅▃▅▄▄▃▂▃▂▁▁,123227100.0,85.5%,+52.5%,"$21,076.74"
2001/06/01,23.0,17843400.0,Missing,█▇▆▄▁▁▁▁▁▁▁▁▁▁▆,123227100.0,14.5%,-18.5%,"$3,568.68"
2001/07/01,17.0,7781400.0,Missing,▁█▄,243448920.0,3.2%,-29.8%,"$1,556.28"
2001/08/01,52.5,192848220.0,Missing,█▂▁▁▁,243448920.0,79.2%,+46.2%,"$38,569.64"
2001/09/01,26.0,42819300.0,Missing,▆▆▁▁▅▆▁██▆▆▆▆,243448920.0,17.6%,-15.4%,"$8,563.86"
2001/10/01,54.0,134975700.0,Missing,▁▁▂▂▃▄▄▅▅▅▆▇▇▇█,480483900.0,28.1%,-4.9%,"$26,995.14"
2001/11/01,104.0,203110200.0,Missing,▆▆▆▆▆▇▇▇██▁▁▁▇▇,480483900.0,42.3%,+9.3%,"$40,622.04"
2001/12/01,115.0,142398000.0,Missing,▆▁▁▁▇▇▁▁▁▆▆▆▁▆▇█,480483900.0,29.6%,-3.4%,"$28,479.60"
2002/01/01,136.0,197745300.0,Missing,██▇███▇▇▁▁▁▁▁▁▇,638525700.0,31.0%,-2.0%,"$39,549.06"
2002/02/01,131.0,157920300.0,Missing,▄▄█▆▆█▁,638525700.0,24.7%,-8.3%,"$31,584.06"


### Psets
With a dataset of the users preference:
1. Color the background of the first two columns blue
2. Format the numeric values by specifying precision and thousands seperator
3. Include a bar plot in a column
4. Set a background gradient for a column
5. Make the column headers sticky

In [84]:
goals_per_yr = (fifa
 .assign(year = lambda df: df.date.dt.year, GD = lambda df: df.home_team_score - df.away_team_score)
 .set_index('date')
 .groupby(pd.Grouper(freq='1Y'))
 .agg(HT_score = ('home_team_score', 'mean'),
      HT_goals = ('home_team_score', 'sum'),
      HT_goal_trend = ('home_team_score', lambda ser: sparklines.sparklines(ser.fillna(0).resample('M').mean().fillna(0))[0]),
      AT_score = ('away_team_score', 'mean'),
      AT_goals = ('away_team_score', 'sum'),
      AT_goal_trend = ('away_team_score', lambda ser: sparklines.sparklines(ser.fillna(0).resample('M').mean().fillna(0))[0]),
      GD = ('GD','sum'),
     )
 .reset_index()
)

In [124]:
(goals_per_yr
 .style
 .format({'date':'{:%Y}',
          **{col:'{:,.1f}' for col in ['HT_score', 'HT_goals','AT_score', 'AT_goals', 'GD']}},
         na_rep='Missing')
#  .highlight_max(axis=0, color='pink', subset=['HT_goals', 'AT_goals'])
#  .highlight_min(axis=0, color='skyblue', subset=['HT_goals', 'AT_goals'])
 .bar(subset=['HT_goals','AT_goals'], color='#c07fef')
 .set_properties(**{'background-color':'orange'},subset=['HT_goal_trend','AT_goal_trend'])
#  .bar(subset='off_goal', color=['red','green'], align='mid')
 .highlight_null(color='#fef70c')
 .background_gradient(axis=0, cmap='Blues', subset='GD', vmin=goals_per_yr.GD.min(), vmax=goals_per_yr.GD.quantile(0.90))
#  .set_caption('Dirty Devil Summary')
 .applymap(lambda val: f'color: "grey"; opacity:80%; background-color:{"lightgreen" if val > goals_per_yr.HT_score.mean() else "lightgray"}', 
            subset='HT_score')
 .set_table_styles([{'selector':'td:hover', 'props':'border:3px solid lightgreen; font-size:10pt;'}])
 .set_sticky(axis="columns")
 .hide(axis='index')
)

date,HT_score,HT_goals,HT_goal_trend,AT_score,AT_goals,AT_goal_trend,GD
1993,1.7,289.0,█▂▄▆▁,1.1,183.0,▃▇█▆▁,106.0
1994,1.4,713.0,▃▁▄▆▇▅▇▅▂█▅▆,1.1,543.0,▅▅▁▄▄▆▅█▅▄▃▄,170.0
1995,1.7,940.0,▁▂▆▄▆▄▆▃█▄▇▅,1.0,590.0,▄▆▃▂▅▅▃█▃▇▁▂,350.0
1996,1.7,"1,321.0",█▆▆▇▄▇▇▆█▁█▄,1.1,879.0,▅▇▃▃▂▄█▁▄▇▅▆,442.0
1997,1.7,"1,379.0",▂▁▆▅▆█▄▄▅▃▅▄,1.1,847.0,▁▄█▃█▆▃▇▆▄▄█,532.0
1998,1.7,"1,078.0",▁▂▆▇▃▆▅▅█▃▂▁,1.1,694.0,▃▅▂▃▂▂▃▁▂▄▄█,384.0
1999,1.7,"1,120.0",▄▁▄▃█▅▃▆▆▂▄▅,1.0,685.0,▁▇▆█▃▄▆▄▆▆▆▃,435.0
2000,1.8,"1,742.0",▇▅▆▆▁▄▆▄▇▃█▂,1.0,993.0,▁▃▃▂▂▃▂▂▁▃▄█,749.0
2001,1.8,"1,759.0",▂▂▂█▆▂▂▃▁▃▂▃,1.2,"1,105.0",▄█▇█▇█▅▇█▄▁▄,654.0
2002,1.5,"1,064.0",▂▃▅▃▃▃█▄▁▄▁▆,1.1,740.0,▁▄▆▂▆▄▂▃▄▄▅█,324.0
